In [45]:
from flask import Flask,request, jsonify
import json
import time
import keras
import numpy as np
app = Flask(__name__)
model = keras.models.load_model("/home/jovyan/dataset/rakuten_model_401_430")
@app.route("/", methods=['GET', 'POST'])
def hello():
    if request.method == 'POST':
        #candidate_list=[]
        can_len = []
        clickstream = []
        sequence_len = []
        data = json.loads(request.data)
        hist_gid = data['hist_g_id']
        candidate_request = data['candidate']
        seq_length = data['seq_length']
        can_len = len(candidate_request)
    
        clickstream+=[hist_gid]*can_len
        sequence_len+=[seq_length]*can_len
            
        candidate_request = np.array(candidate_request)
        clickstream = np.array(clickstream)
        sequence_len = np.array(sequence_len)
        
        x_pred = {'g_id': candidate_request, 
                'hist_g_id': clickstream, 
                'seq_length': sequence_len}
        print(x_pred)
        y_pred = model.predict(x_pred)
        y_pred_score = y_pred.reshape(-1)
        y_pred_score = y_pred_score.tolist()
        
        
        rerank_list = []
        tempdict = {}
        for i in zip(candidate_request,y_pred_score):
            tempdict[i[0]] = i[1]
        tempdict = {k: v for k, v in sorted(tempdict.items(), key=lambda item: item[1], reverse=True)}
        rerank_list.append(list(tempdict.keys()))
        respond = {'rerank':rerank_list}
        
        def convert(o):
            if isinstance(o, np.generic): return o.item()  
            raise TypeError
        return json.dumps(respond, default=convert)
        
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


{'g_id': array([1046873,  798421,  990655,  226697,   23467, 1333060, 1030811,
         94341, 1539411, 1534230,  757623, 1519174, 1839503,  898817,
        223846, 1598160,  555647, 1846256,  232675, 1264920,  686272,
        786385, 1028995, 1313552, 1076777, 1029365,  368416,  531902,
       1326280,  436448]), 'hist_g_id': array([[1315304,       0,       0, ...,       0,       0,       0],
       [1315304,       0,       0, ...,       0,       0,       0],
       [1315304,       0,       0, ...,       0,       0,       0],
       ...,
       [1315304,       0,       0, ...,       0,       0,       0],
       [1315304,       0,       0, ...,       0,       0,       0],
       [1315304,       0,       0, ...,       0,       0,       0]]), 'seq_length': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])}


127.0.0.1 - - [13/Sep/2021 13:17:14] "POST / HTTP/1.1" 200 -
